1. regular prediction
2. random columns prediction

3. for dist in (wesserstain, hellinger, JM):
    1. ranked columns prediction
    2. ranked + kmeans precition
    3. mediods prediction

In [1]:
from math import exp, sqrt, log
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

from dictances import jensen_shannon
import matplotlib.pyplot as plt
from pydiffmap import diffusion_map as dm
from pydiffmap.visualization import embedding_plot, data_plot

from ref.Diffusion_Maps import diffusionMapping
from ref.Shir import utils as shir_utils
from utils import min_max_scaler, calc_mean_std, flatten, norm_by_dist_type, calculate_distance
from main import execute_distance_func, calc_dist, export_heatmaps, k_medoids_features, return_best_features_by_kmeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
df = pd.read_csv('data/glass.csv')
features = df.columns.drop('label')

In [9]:
def predict(df, features, label_column):
    X_train, X_test, y_train, y_test = train_test_split(df[features], df[label_column], test_size=0.33, random_state=42)
    forest = RandomForestClassifier(random_state=1)
    multi_target_forest = OneVsRestClassifier(forest, n_jobs=-1)
    model = multi_target_forest.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    print(metrics.classification_report(y_test, preds, digits=3))
    return preds

In [10]:
preds = predict(df, features, 'label')

              precision    recall  f1-score   support

           1      0.769     0.909     0.833        22
           2      0.792     0.760     0.776        25
           3      1.000     0.500     0.667         4
           5      1.000     0.500     0.667         6
           6      0.800     1.000     0.889         4
           7      0.909     1.000     0.952        10

    accuracy                          0.817        71
   macro avg      0.878     0.778     0.797        71
weighted avg      0.831     0.817     0.809        71



# ***Random columns

In [17]:
sampled_df = df[features].sample(n=4, axis='columns')
new_features = sampled_df.columns
sampled_df['label'] = df['label']
preds = predict(sampled_df, new_features, 'label')

              precision    recall  f1-score   support

           1      0.704     0.864     0.776        22
           2      0.720     0.720     0.720        25
           3      0.000     0.000     0.000         4
           5      0.250     0.167     0.200         6
           6      0.600     0.750     0.667         4
           7      0.800     0.800     0.800        10

    accuracy                          0.690        71
   macro avg      0.512     0.550     0.527        71
weighted avg      0.639     0.690     0.661        71



C:\Users\OmerHedvat\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
new_featurestures

Index(['feature_7', 'feature_2', 'feature_3', 'feature_8', 'feature_4',
       'feature_5'],
      dtype='object')